In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/friday-workinghours-afternoon/Friday-WorkingHours-Afternoon-D.csv


In [55]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix
import time

In [56]:
data=pd.read_csv('/kaggle/input/friday-workinghours-afternoon/Friday-WorkingHours-Afternoon-D.csv')


In [57]:
data

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225740,61374,61,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225741,61378,72,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225742,61375,75,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225743,61323,48,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [58]:
data[' Label'].value_counts()

 Label
DDoS      128027
BENIGN     97718
Name: count, dtype: int64

In [59]:
data.duplicated().sum()

2633

In [60]:
data = data.drop_duplicates()

In [61]:
data.dtypes

 Destination Port                int64
 Flow Duration                   int64
 Total Fwd Packets               int64
 Total Backward Packets          int64
Total Length of Fwd Packets      int64
                                ...   
Idle Mean                      float64
 Idle Std                      float64
 Idle Max                        int64
 Idle Min                        int64
 Label                          object
Length: 79, dtype: object

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 223112 entries, 0 to 225744
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             223112 non-null  int64  
 1    Flow Duration                223112 non-null  int64  
 2    Total Fwd Packets            223112 non-null  int64  
 3    Total Backward Packets       223112 non-null  int64  
 4   Total Length of Fwd Packets   223112 non-null  int64  
 5    Total Length of Bwd Packets  223112 non-null  int64  
 6    Fwd Packet Length Max        223112 non-null  int64  
 7    Fwd Packet Length Min        223112 non-null  int64  
 8    Fwd Packet Length Mean       223112 non-null  float64
 9    Fwd Packet Length Std        223112 non-null  float64
 10  Bwd Packet Length Max         223112 non-null  int64  
 11   Bwd Packet Length Min        223112 non-null  int64  
 12   Bwd Packet Length Mean       223112 non-null  fl

In [63]:
data.isnull().sum()

 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64

In [64]:
data.describe()

/opt/conda/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/opt/conda/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,223112.000000,2.231120e+05,223112.000000,223112.000000,223112.000000,2.231120e+05,223112.000000,223112.000000,223112.000000,223112.000000,...,223112.000000,223112.000000,2.231120e+05,2.231120e+05,2.231120e+05,2.231120e+05,2.231120e+05,2.231120e+05,2.231120e+05,2.231120e+05
mean,8982.617537,1.643322e+07,4.905375,4.611554,949.663380,6.029361e+03,544.576002,27.901726,166.459564,217.439495,...,3.335988,21.434921,1.870073e+05,1.308700e+04,2.105405e+05,1.797162e+05,1.044389e+07,3.654568e+06,1.303004e+07,7.846807e+06
std,19847.531015,3.166018e+07,15.509753,21.880161,3267.081005,3.944391e+04,1874.258315,164.242839,507.624778,801.759676,...,12.338457,4.099242,8.023654e+05,2.115061e+05,9.052459e+05,7.886355e+05,2.195266e+07,1.282588e+07,2.704309e+07,1.992977e+07
min,0.000000,-1.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,80.000000,8.163775e+04,2.000000,1.000000,26.000000,0.000000e+00,6.000000,0.000000,6.000000,0.000000,...,1.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,80.000000,1.536448e+06,3.000000,4.000000,30.000000,1.640000e+02,20.000000,0.000000,8.666667,5.301991,...,2.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,80.000000,8.957830e+06,5.000000,5.000000,62.000000,1.160100e+04,35.000000,6.000000,32.000000,10.263203,...,4.000000,20.000000,1.972000e+03,0.000000e+00,1.972000e+03,1.966000e+03,8.418146e+06,0.000000e+00,8.444972e+06,7.524811e+06
max,65532.000000,1.199999e+08,1932.000000,2942.000000,183012.000000,5.172346e+06,11680.000000,1472.000000,3867.000000,6692.644993,...,1931.000000,52.000000,1.000000e+08,3.950000e+07,1.000000e+08,1.000000e+08,1.200000e+08,6.530000e+07,1.200000e+08,1.200000e+08


In [65]:
data[' Label'].value_counts()

 Label
DDoS      128016
BENIGN     95096
Name: count, dtype: int64

In [66]:
data[' Label'] = data[' Label'].replace('BENIGN', 0)
data[' Label'] = data[' Label'].replace('DDoS', 1)

/tmp/ipykernel_33/3217156119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[' Label'] = data[' Label'].replace('BENIGN', 0)
/tmp/ipykernel_33/3217156119.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[' Label'] = data[' Label'].replace('DDoS', 1)
/tmp/ipykernel_33/3217156119.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [67]:
data

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225740,61374,61,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
225741,61378,72,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
225742,61375,75,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
225743,61323,48,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [68]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)

/tmp/ipykernel_33/3102627609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.replace([np.inf, -np.inf], np.nan, inplace=True)


In [69]:
#dropping the null values
data.dropna(inplace=True)

/tmp/ipykernel_33/1349455580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [70]:
data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [71]:
#Dividing in X and y
X = data.drop(' Label', axis=1)
y = data[' Label']

In [72]:
#Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [73]:
#Normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [74]:
# Define the SVM model
svm = SVC()


In [79]:
# Parameter grid for Grid Search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'gamma': ['scale', 'auto']   
}

In [80]:
# Perform Grid Search
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear']},
             scoring='accuracy')

In [81]:
# Get the best parameters and model from Grid Search
best_params_grid = grid_search.best_params_
best_model_grid = grid_search.best_estimator_

In [88]:
# Parameter grid for Random Search
param_dist = {
    'C': np.logspace(-3, 2,5),
    'kernel': ['linear'],
    'gamma': ['scale', 'auto']
}


In [89]:
# Perform Random Search
random_search = RandomizedSearchCV(estimator=svm, param_distributions=param_dist, n_iter=3, scoring='accuracy', cv=2, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=2, estimator=SVC(), n_iter=3, n_jobs=-1,
                   param_distributions={'C': array([1.00000000e-03, 1.77827941e-02, 3.16227766e-01, 5.62341325e+00,
       1.00000000e+02]),
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['linear']},
                   random_state=42, scoring='accuracy')

In [90]:
# Get the best parameters and model from Random Search
best_params_random = random_search.best_params_
best_model_random = random_search.best_estimator_


In [91]:
# Evaluate the best model from Grid Search
start_time = time.time()
y_pred_grid = best_model_grid.predict(X_test)
prediction_time_grid = time.time() - start_time
accuracy_grid = accuracy_score(y_test, y_pred_grid)
precision_grid = precision_score(y_test, y_pred_grid, average='weighted')
recall_grid = recall_score(y_test, y_pred_grid, average='weighted')
f1_grid = f1_score(y_test, y_pred_grid, average='weighted')
conf_matrix_grid = confusion_matrix(y_test, y_pred_grid)
ckc_grid = cohen_kappa_score(y_test, y_pred_grid)

In [92]:
# Calculate FAR
far_grid = conf_matrix_grid[0,1] / np.sum(conf_matrix_grid[:,1])

In [93]:
# Evaluate the best model from Random Search
start_time = time.time()
y_pred_random = best_model_random.predict(X_test)
prediction_time_random = time.time() - start_time
accuracy_random = accuracy_score(y_test, y_pred_random)
precision_random = precision_score(y_test, y_pred_random, average='weighted')
recall_random = recall_score(y_test, y_pred_random, average='weighted')
f1_random = f1_score(y_test, y_pred_random, average='weighted')
conf_matrix_random = confusion_matrix(y_test, y_pred_random)
ckc_random = cohen_kappa_score(y_test, y_pred_random)

In [94]:
# Calculate FAR
far_random = conf_matrix_random[0,1] / np.sum(conf_matrix_random[:,1])


In [98]:
# Print the results
print("Grid Search Best Parameters:")
print(best_params_grid)
print("\nRandom Search Best Parameters:")
print(best_params_random)
print("\nPerformance Metrics for Grid Search:")
print("Accuracy:",accuracy_grid)
print("Precision:", precision_grid)
print("Recall:", recall_grid)
print("F1 Score:", f1_grid)
print("Cohen's Kappa Coefficient:", ckc_grid)
print("Prediction Time:", prediction_time_grid)
print("False Alarm Rate (FAR):", far_grid)
print("\nPerformance Metrics for Random Search:")
print("Accuracy:",accuracy_random)
print("Precision:", precision_random)
print("Recall:", recall_random)
print("F1 Score:", f1_random)
print("Cohen's Kappa Coefficient:", ckc_random)
print("Prediction Time:", prediction_time_random)
print("False Alarm Rate (FAR):", far_random)

Grid Search Best Parameters:
{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}

Random Search Best Parameters:
{'kernel': 'linear', 'gamma': 'scale', 'C': 100.0}

Performance Metrics for Grid Search:
Accuracy: 0.99
Precision: 0.991567813348202
Recall: 0.9914558939389824
F1 Score: 0.9914449039067965
Cohen's Kappa Coefficient: 0.9824933580654653
Prediction Time: 10.856845617294312
False Alarm Rate (FAR): 0.014320422370695697

Performance Metrics for Random Search:
Accuracy: 0.9968757640794371
Precision: 0.9968861272758006
Recall: 0.9968757640794371
F1 Score: 0.9968745724769784
Cohen's Kappa Coefficient: 0.9936093020183498
Prediction Time: 9.49664306640625
False Alarm Rate (FAR): 0.0048828816078126105
